In [1]:
from manim import *

Manim Community v0.18.0

In [2]:
import sys
import os

# Configuración del directorio raíz del proyecto
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Directorio raíz
# Configuración de las rutas
sys.path.append(os.path.join(project_root, 'src'))  # Agregar 'src' al path para importaciones

In [ ]:
# config.media_width = "75%"
# config.verbosity = "WARNING"
# config.background_color = "#24043d"

In [ ]:
import funciones as f
# f.etiquetar_ejes()

In [ ]:
import herramientas as h

In [ ]:
%%writefile Intro.py
from manim import *
import numpy as np

# Configuración vertical (1080x1920)
config.pixel_width = 1080
config.pixel_height = 1920
config.frame_width = 5.625
config.frame_height = 10.0
config.background_color = BLACK
config.media_dir = "./media"  # Directorio donde se guarda el video
#config.background_color = "#24043d"

# Función auxiliar para grilla (igual que antes)
def en_grilla(grupo: VGroup, columnas: int, espacio_x: float = 1.0, espacio_y: float = 1.0,
              alineacion_horizontal: str = "centro", alineacion_vertical: str = "centro",
              max_izquierda: float = -6, max_derecha: float = 6,
              max_abajo: float = -3.5, max_arriba: float = 3.5,
              offset_x: float = 0, offset_y: float = 0) -> VGroup:
    n = len(grupo)
    filas = (n + columnas - 1) // columnas
    for idx, obj in enumerate(grupo):
        fila = idx // columnas
        col = idx % columnas
        obj.move_to([col * espacio_x, -fila * espacio_y, 0])
    ancho_total = (min(columnas, n) - 1) * espacio_x
    alto_total = (filas - 1) * espacio_y
    if alineacion_horizontal == "centro":
        grupo.shift(RIGHT * ((max_derecha + max_izquierda - ancho_total) / 2 + offset_x))
    elif alineacion_horizontal == "izquierda":
        grupo.shift(LEFT * abs(max_izquierda) + RIGHT * offset_x)
    elif alineacion_horizontal == "derecha":
        grupo.shift(RIGHT * abs(max_derecha - ancho_total) + RIGHT * offset_x)
    if alineacion_vertical == "centro":
        grupo.shift(UP * ((max_arriba + max_abajo - alto_total) / 2 + offset_y))
    elif alineacion_vertical == "arriba":
        grupo.shift(UP * abs(max_arriba) + UP * offset_y)
    elif alineacion_vertical == "abajo":
        grupo.shift(DOWN * abs(max_abajo - alto_total) + UP * offset_y)
    return grupo

def entrada_con_rebote(objeto, destino, desde=None, altura_rebote=1.5, duracion_total=2.5):
    if desde is None:
        desde = destino + 5 * UP

    objeto.move_to(desde)
    desplazamiento_total = desde - destino

    dur_caida = duracion_total * 0.5
    dur_transicion = duracion_total * 0.1
    dur_rebote = duracion_total - dur_caida - dur_transicion

    # Animación 1: caída suave (ease-in)
    def anim_caida_fn(mob, alpha):
        y_offset = 1 - (1 - alpha) ** 2  # aceleración suave
        mob.move_to(destino + (1 - y_offset) * desplazamiento_total)

    caida = UpdateFromAlphaFunc(objeto, anim_caida_fn, run_time=dur_caida, rate_func=linear)

    # Punto inicial de rebote
    rebote_pico = destino + altura_rebote * UP

    # Animación 2: transición rápida del destino al pico del rebote
    transicion = objeto.animate.move_to(rebote_pico).set_run_time(dur_transicion).set_rate_func(smooth)

    # Animación 3: rebote clásico hacia abajo
    def curva_rebote_clasica(t):
        if t < 0.3636:
            y = 7.5625 * t * t
        elif t < 0.7272:
            t -= 0.5454
            y = 7.5625 * t * t + 0.75
        elif t < 0.9090:
            t -= 0.8181
            y = 7.5625 * t * t + 0.9375
        else:
            t -= 0.9545
            y = 7.5625 * t * t + 0.984375
        return y

    def anim_rebote_fn(mob, alpha):
        y_offset = curva_rebote_clasica(alpha)
        mob.move_to(destino + (1 - y_offset) * altura_rebote * UP)

    rebote = UpdateFromAlphaFunc(objeto, anim_rebote_fn, run_time=dur_rebote, rate_func=linear)

    return Succession(caida, transicion, rebote)

def desaparecer_gradualmente(mob, alpha):
      mob.set_opacity(1 - alpha)


def etiquetar_ejes(ejes: ThreeDAxes, escala: float = 0.65, color: str = WHITE) -> VGroup:
    """Devuelve un grupo con las etiquetas 'X', 'Y', 'Z' posicionadas al final de cada eje."""
    etiqueta_x = Text("X", color=color).scale(escala)
    etiqueta_y = Text("Y", color=color).scale(escala)
    etiqueta_z = Text("Z", color=color).scale(escala)

    etiqueta_x.move_to(ejes.x_axis.get_end() + RIGHT * 0.3)
    etiqueta_y.move_to(ejes.y_axis.get_end() + UP * 0.3)
    etiqueta_z.move_to(ejes.z_axis.get_end() + OUT * 0.3)  # OUT es hacia la cámara

    return VGroup(etiqueta_x, etiqueta_y, etiqueta_z)


# Escena vertical 3d
class Intro(ThreeDScene):
    def construct(self):

        logo = ImageMobject("logorobo.png").set_opacity(0.55)  # Asegúrate de que esté en el mismo directorio
        logo.scale(0.45)  # Opcional: escala la imagen
        logo.to_corner(DL).shift(0.54*LEFT + 0.6*DOWN)  # Coloca la imagen en la esquina superior derecha
        #self.add(logo)

        # ejes = ThreeDAxes()
        # ejes.scale(0.8)
        # self.add(ejes)
        # etiquetas = etiquetar_ejes(ejes)
        # self.add(etiquetas)

        rectangulos = VGroup(*[
            Rectangle(width=1.8, height=2.6, color=WHITE).scale(0.703125)
            for _ in range(20)
        ])

        en_grilla(
            rectangulos,
            columnas=4,
            espacio_x=2 * 0.7,        # Debería ser * 0.703125 pero la diferencia es despreciable
            espacio_y=2.8 * 0.7,
            alineacion_horizontal="centro",
            alineacion_vertical="centro",
            max_izquierda=-2.8125,    # suman el ancho del frame
            max_derecha=2.8125,       #
            max_abajo=2.85,           # Estos dos tienen una distancia igual al alto del frame,
            max_arriba=12.85          # pero corridos 2.85 unidades hacia arriba
        )                             # por el offset de la función, en vertical.


        #self.play(FadeIn(rectangulos))
        glow_marco = RoundedRectangle(
            width=4.92,          # Ajustá según la proporción visible del marco interior
            height=7.12,          # Idem
            corner_radius=0.8  # Suavidad de las esquinas
        )
        glow_marco.set_stroke(color=BLUE, width=40, opacity=1)  # Azul turquesa con efecto suave
        glow_marco.set_fill(opacity=0)  # Solo borde
        glow_marco.move_to(ORIGIN).shift(0.02*DOWN)      # Asegurarse que quede alineado

        # Glow interno
        glow_marco2 = RoundedRectangle(
            width=4.54,          # Ajustá según la proporción visible del marco interior
            height=6.75,          # Idem
            corner_radius=0.8  # Suavidad de las esquinas
        )
        glow_marco2.set_stroke(color=BLUE, width=10, opacity=0.9)  # Azul turquesa con efecto suave
        glow_marco2.set_fill(opacity=0)  # Solo borde
        glow_marco2.move_to(ORIGIN).shift(0.04*DOWN)

        compu = ImageMobject("compu_v_color.png")
        compu.scale(0.98)
        grupo_intro = Group()
        titulo = Text("Los 10 materiales\nmás caros del mundo!", color = WHITE)
        titulo.next_to(rectangulos[4], RIGHT, buff=0).shift(1.4*LEFT) #1/2 del espacio entre centros de rectángulos, en x
        compu = ImageMobject("compu_v_color.png").scale(0.98)

        grupo_intro.add(glow_marco)
        grupo_intro.add(compu)
        grupo_intro.add(glow_marco2)

        titulo.scale(0.7)

        self.play(entrada_con_rebote(grupo_intro, destino=ORIGIN, desde = 10 * UP, altura_rebote=0.6,
                                     duracion_total=1.4))
        #self.add_fixed_orientation_mobjects(titulo)
        self.play(Write(titulo))
        grupo_intro.add(titulo)

        self.play(
                AnimationGroup(
                    grupo_intro.animate.scale(1.6),
                    UpdateFromAlphaFunc(titulo, desaparecer_gradualmente),
                    FadeIn(logo),
                    lag_ratio=0.15  # Ajustá este valor a gusto (0 = todo a la vez, 1 = secuencial)
                ),
                run_time=2.5
            )

        self.wait(1)
